In [5]:
import cv2

## Utilities

In [6]:
def getFromFile(file,info):
    cap = cv2.VideoCapture(file)
    ret, frame = cap.read()
    cap.release()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if(info):
        print(f'The shape of the image is {frame_rgb.shape}')
    return frame_rgb

In [7]:
def MaskImg(file,corners):
    image = getFromFile(file,0)
    mask = np.zeros(image.shape[:2], dtype=np.uint8)#CHIEDI
    mask_corners = np.array(corners)
    cv2.drawContours(mask, [mask_corners], -1, 255, -1)
#     plt.imshow(mask, cmap='gray')
    masked_image = cv2.bitwise_and(image,image,mask=mask)
    plt.imshow(masked_image)
    return masked_image

In [4]:
#get a single fram from a video
def getFrame(file,num):
    cap = cv2.VideoCapture(file)
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f'The video contains {n_frames} frames')
    if(num>=n_frames):
        print("FRAME OUT OF INDEX")
        return -1
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, num)
        ret, frame = cap.read()
        cap.release()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame_rgb

## Convert bgr to hsv

In [1]:
#this functions converts rgb into hsv format and returns two arrays: the lower and the upper limit to apply to the mask
def getLimits(b,g,r,rangeWidth):
    arr = [[[b,g,r]]]
    color = np.uint8(arr)
    hsvColor = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)
    lowerLimit = [hsvColor[0][0][0]-rangeWidth,40,40]
    upperLimit = [hsvColor[0][0][0]+rangeWidth,255,255]
    return [lowerLimit,upperLimit]

## object tracking

In [2]:
#track a falling object in a video
def track(r,g,b,width,file,outfilename,corners):
    cap = cv2.VideoCapture(file)
    #get video information
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
# outfilename = "tracked.mp4"   

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(outfilename, fourcc, fps, (frame_width, frame_height))
    #fix pyplot axes
    plt.axis([0,780,0,1280])

    for frame_index in range(n_frames):
        ret, frame_rgb = cap.read()
        #apply mask given the corners (passed as a parameter)
        mask = np.zeros(frame_rgb.shape[:2], dtype=np.uint8)#CHIEDI
        mask_corners = np.array(corners)
        cv2.drawContours(mask, [mask_corners], -1, 255, -1)
        masked_image = cv2.bitwise_and(frame_rgb,frame_rgb,mask=mask)

        hsv_image =  cv2.cvtColor(masked_image, cv2.COLOR_BGR2HSV)

        blue_regions = cv2.inRange(hsv_image, np.array(lowerLimit), np.array(upperLimit))#lower and upper limit are to be enclosed in an np array

        contours, _ = cv2.findContours(blue_regions, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#CHIEDI
        blue_dots_positions = []

        for contour in contours:
    #       print(f"area = {cv2.contourArea(contour)}")
          if cv2.contourArea(contour) > 100: # Minimum contour area threshold
            M = cv2.moments(contour)
    #         print(M)
            center_x = M["m10"] / M["m00"]
            center_y = M["m01"] / M["m00"]#CHIEDI
            print(f"Centroid of dot: (x, y) = ({center_x:.1f}, {center_y:.1f})")
    
            plt.scatter(center_x, center_y, color='blue', marker='o', facecolor='none', s=200);#s is the size of the scattered contour
            blue_dots_positions.append([center_x, center_y])
            #draw circle around the found position
            cv2.circle(frame_rgb,(int(center_x),int(center_y)),30,(0,0,0),5)
        out.write(frame_rgb)
        
    cap.release()
    out.release()
    print(f'Video written to {outfilename}')

In [3]:
def frameCenter(frame,r,g,b,width,area):
    lims=getLimits(b,g,r,width)
    lowerLimit = lims[0]
    upperLimit = lims[1]
    hsv_image =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    blue_regions = cv2.inRange(hsv_image, np.array(lowerLimit), np.array(upperLimit))#lower and upper limit are to be enclosed in an np array
    
    contours, _ = cv2.findContours(blue_regions, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#CHIEDI
#     blue_dots_positions = []

    for contour in contours:
#       print(f"area = {cv2.contourArea(contour)}")
      if cv2.contourArea(contour) > area: # Minimum contour area threshold
        M = cv2.moments(contour)
        center_x = M["m10"] / M["m00"]
        center_y = M["m01"] / M["m00"]#CHIEDI
        print(f"Centroid of dot: (x, y) = ({center_x:.1f}, {center_y:.1f})")
#         plt.scatter(center_x, center_y, color='blue', marker='o', facecolor='none', s=200);#s is the size of the scattered contour
#         blue_dots_positions.append([center_x, center_y])
        return [center_x,center_y]